In [9]:
# CÉLULA 1: INSTALAÇÃO E CONFIGURAÇÃO DO FLASK E NGROK
# -------------------------------------------------
# Instala o Flask para criar o servidor web e o ngrok para criar um túnel público.
# Isso permite que a sua interface HTML no navegador se comunique com o Python no Colab.
!pip install Flask flask-cors pyngrok -q

# IMPORTANTE: Você precisa de um token de autenticação do ngrok.
# 1. Visite https://ngrok.com/signup e crie uma conta gratuita.
# 2. No seu dashboard ngrok, copie seu token de autenticação (Auth token).
# 3. Cole-o abaixo, substituindo "SEU_NGROK_AUTH_TOKEN_AQUI".
from pyngrok import ngrok
import os

# VVVV SUBSTITUA "SEU_NGROK_AUTH_TOKEN_AQUI" PELO SEU TOKEN VVVV
ngrok_auth_token = "2yYlDIdHTSU0RGwOdgdiHkGsezW_3V5516xVoSQeuUdurqcSG"
# ^^^^ SUBSTITUA "SEU_NGROK_AUTH_TOKEN_AQUI" PELO SEU TOKEN ^^^^

if ngrok_auth_token == "SEU_NGROK_AUTH_TOKEN_AQUI":
    print("⚠️ ATENÇÃO: Por favor, insira seu token de autenticação do ngrok na Célula 1 deste Canvas.")
else:
    ngrok.set_auth_token(ngrok_auth_token)
    print("Token ngrok configurado.")

# Importa as bibliotecas necessárias para o agente de análise de notas fiscais
# As mesmas instalações e configurações da Célula 1 e 2 do seu notebook original
# são replicadas aqui para garantir que este servidor seja autocontido.
!pip install langchain langchain_experimental langchain-community pandas tabulate gdown langchain-google-genai -q

# Configuração da Chave de API do Google Generative AI para o servidor Flask
# Certifique-se que esta chave é a mesma que você usa no seu outro Canvas.
os.environ['GOOGLE_API_KEY'] = "OPENAI_API_KEY" # <<<<<< SUBSTITUA PELA SUA CHAVE DE API AQUI!

Token ngrok configurado.


In [10]:
# CÉLULA 2: DOWNLOAD E PREPARAÇÃO DOS DADOS (REPETIÇÃO DO SEU CÓDIGO ORIGINAL)
# -------------------------------------------------
# Esta seção é uma cópia das Células 3, 4 e 5 do seu notebook original.
# O servidor precisa ter acesso aos mesmos dados que o agente.
import gdown
import zipfile
import pandas as pd

google_drive_file_id = "1jaNl2kL-a32HO_jXH3g3J1mGHzHWm9YD" # <<<< MANTENHA O MESMO ID DO SEU ARQUIVO ORIGINAL
zip_file_name = '202401_NFs.zip'
output_path = f'./{zip_file_name}'

if google_drive_file_id == "SEU_FILE_ID_AQUI":
    print("⚠️ ATENÇÃO: Por favor, substitua 'SEU_FILE_ID_AQUI' pelo ID do seu arquivo do Google Drive na Célula 2 deste Canvas.")
else:
    url = f'https://drive.google.com/uc?id={google_drive_file_id}'
    print(f"Baixando o arquivo de ID: {google_drive_file_id}...")
    try:
        gdown.download(url, output_path, quiet=False)

        if os.path.exists(zip_file_name):
            print(f"\nDownload concluído: {zip_file_name}")
            with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
                zip_ref.extractall('dados_nfs')
            print("\nArquivos descompactados com sucesso na pasta 'dados_nfs':")
            !ls dados_nfs
        else:
            print("\n❌ ERRO: O download do arquivo falhou. Verifique o ID do arquivo e as permissões.")

        # Carregamento dos dados com Pandas
        caminho_cabecalho = 'dados_nfs/202401_NFs_Cabecalho.csv'
        caminho_itens = 'dados_nfs/202401_NFs_Itens.csv'

        try:
            df_cabecalho = pd.read_csv(caminho_cabecalho, sep=',')
            df_itens = pd.read_csv(caminho_itens, sep=',')
            df_completo = pd.merge(df_itens, df_cabecalho, on='id_nota_fiscal', how='left')
            print("\n✅ Dados carregados e combinados com sucesso no servidor.")
        except FileNotFoundError:
            print("\nERRO: Um ou ambos os arquivos CSV não foram encontrados no servidor.")
            df_completo = pd.DataFrame() # Cria um DataFrame vazio para evitar erros
    except Exception as e:
        print(f"Erro ao baixar ou processar os dados: {e}")
        df_completo = pd.DataFrame() # Cria um DataFrame vazio em caso de erro no download


Baixando o arquivo de ID: 1jaNl2kL-a32HO_jXH3g3J1mGHzHWm9YD...


Downloading...
From: https://drive.google.com/uc?id=1jaNl2kL-a32HO_jXH3g3J1mGHzHWm9YD
To: /content/202401_NFs.zip
100%|██████████| 42.9k/42.9k [00:00<00:00, 50.3MB/s]


Download concluído: 202401_NFs.zip

Arquivos descompactados com sucesso na pasta 'dados_nfs':
202401_NFs_Cabecalho.csv  202401_NFs_Itens.csv
Erro ao baixar ou processar os dados: 'id_nota_fiscal'


In [ ]:
# CÉLULA 3: CRIAÇÃO DO AGENTE E SERVIDOR FLASK
# -------------------------------------------------
# Configura o agente e inicia o servidor Flask.
from flask import Flask, request, jsonify
from flask_cors import CORS
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

# Move a criação da instância do Flask e as definições de rota para uma função
# para garantir que o contexto seja criado apenas quando o servidor for executado.
def create_flask_app(dataframe): # df_completo agora é passado como argumento
    app = Flask(__name__)
    CORS(app) # Permite requisições de diferentes origens (para o HTML)

    # Configura a chave de API (necessário para o agente funcionar)
    genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

    # Inicializa o modelo de linguagem (Gemini)
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.1)

    # Cria o agente de análise de dados (movido para dentro da função e usa 'dataframe')
    agent = create_pandas_dataframe_agent(
        llm,
        dataframe, # Usa o dataframe passado como argumento
        verbose=True,
        handle_parsing_errors=True,
        allow_dangerous_code=True
    )

    @app.route("/")
    def home():
        return "Servidor Flask para o Agente de Análise de Notas Fiscais está ativo!"

    @app.route("/ask_agent", methods=["POST"])
    def ask_agent():
        data = request.json
        question = data.get("question")

        if not question:
            return jsonify({"error": "Pergunta não fornecida."}), 400

        try:
            print(f"\nRecebida pergunta: {question}")
            # Invoca o agente com a pergunta do usuário
            response = agent.invoke(question)
            answer = response.get('output', "Desculpe, não consegui gerar uma resposta.")
            print(f"Resposta do agente: {answer}")
            return jsonify({"answer": answer})
        except Exception as e:
            print(f"Erro ao invocar o agente: {e}")
            return jsonify({"error": f"Ocorreu um erro ao processar sua pergunta: {str(e)}"}), 500
    return app

# Inicia o túnel ngrok e o servidor Flask
# Nota: Esta célula bloqueará a execução.
# Você precisará executar esta célula e, em seguida, acessar a URL pública fornecida pelo ngrok.
try:
    public_url = ngrok.connect(5000).public_url
    print(f"🌟 Túnel ngrok estabelecido! Seu servidor Flask está acessível em: {public_url}")
    print("Copie esta URL e cole na variável 'backendApiUrl' no seu arquivo HTML.")

    # Cria a instância do aplicativo aqui antes de executar, passando df_completo
    app_instance = create_flask_app(df_completo)
    # Adicionado debug=False e use_reloader=False para estabilidade no Colab
    app_instance.run(port=5000, debug=False, use_reloader=False)
except Exception as e:
    print(f"❌ ERRO ao iniciar ngrok ou Flask: {e}")
    print("Verifique se o seu token ngrok está correto e se a porta 5000 está livre.")

🌟 Túnel ngrok estabelecido! Seu servidor Flask está acessível em: https://b8f0-34-55-229-121.ngrok-free.app
Copie esta URL e cole na variável 'backendApiUrl' no seu arquivo HTML.
 * Serving Flask app '__main__'
 * Debug mode: off


/usr/local/lib/python3.11/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
